<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Leitura-dos-dados-de-chamadas" data-toc-modified-id="Leitura-dos-dados-de-chamadas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Leitura dos dados de chamadas</a></span><ul class="toc-item"><li><span><a href="#Coluna-1:-faixa-de-hora-no-dia" data-toc-modified-id="Coluna-1:-faixa-de-hora-no-dia-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Coluna 1: faixa de hora no dia</a></span></li><li><span><a href="#Coluna-2:-tipo-de-dia" data-toc-modified-id="Coluna-2:-tipo-de-dia-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Coluna 2: tipo de dia</a></span></li><li><span><a href="#Coluna-3:-bairro" data-toc-modified-id="Coluna-3:-bairro-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Coluna 3: bairro</a></span></li><li><span><a href="#Coluna-4:-prioridade-da-chamada" data-toc-modified-id="Coluna-4:-prioridade-da-chamada-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Coluna 4: prioridade da chamada</a></span></li><li><span><a href="#Coluna-5:-ocorrência" data-toc-modified-id="Coluna-5:-ocorrência-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Coluna 5: ocorrência</a></span></li></ul></li><li><span><a href="#Criar-classe-do-modelo" data-toc-modified-id="Criar-classe-do-modelo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Criar classe do modelo</a></span></li><li><span><a href="#Treinar-por-100-epochs" data-toc-modified-id="Treinar-por-100-epochs-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Treinar por 100 epochs</a></span></li><li><span><a href="#Avaliar-evolução-dos-parâmetros" data-toc-modified-id="Avaliar-evolução-dos-parâmetros-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Avaliar evolução dos parâmetros</a></span></li></ul></div>

In [ ]:
import sys
import torch
import numpy as np  

from tqdm import tqdm

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

import matplotlib.pyplot as plt

In [ ]:
sys.path.append('../../..')
from config import *
from utils import *

# Leitura dos dados

In [ ]:
df = pd.read_pickle(os.path.join(TRTD_DATA_PATH,'eventos.pkl'))
df.head()

In [ ]:
vizinhancas = pd.read_csv(os.path.join(ENTR_DATA_PATH,'bairros_vizinhos.csv'),sep=';',encoding='latin-1',index_col=0)
vizinhancas.head()

# Tratamentos

In [ ]:
# dropar paquetá
vizinhancas.drop(0,inplace=True)
df = df[df['i']!=0]

In [ ]:
bairro_to_idx = df[['i','nome_bairro']].drop_duplicates().set_index('nome_bairro')['i'].to_dict()

vizinhancas['NOME'] = vizinhancas['NOME'].map(bairro_to_idx)
bairro_to_vizinhos = vizinhancas.set_index('NOME')['NEIGHBORS'].apply(lambda v:[bairro_to_idx[b.strip()] for b in v.split(',')]).to_dict()

In [ ]:
idx_to_i = {idx:i for (i,idx) in enumerate(bairro_to_vizinhos.keys())}
i_to_idx = {i:idx for (i,idx) in enumerate(bairro_to_vizinhos.keys())}

In [ ]:
N_BAIRROS = len(bairro_to_idx)

In [ ]:
p = 1
g = 1
t = 24

Y = np.zeros(shape=(N_BAIRROS,522))

for bairro,idx in tqdm(list(bairro_to_idx.items())):
    i = idx_to_i[idx]
    Y[i,:] = calcular_distribuicao(df,t,g,idx,p,tipo_discretizacao_temp=0)